# Some statistics for models

## ResNet

Number of trainable parameters for each ResNet depth, IE. the number of FLOPs (Floating Point Operations), which directly corresponds to complexity.

In [4]:
from models.resnet import ResNet, ResidualBlock

resnet18 = ResNet(12, 5, 18, ResidualBlock)
resnet34 = ResNet(12, 5, 34, ResidualBlock)

In [5]:
resnet18_params = sum(p.numel() for p in resnet18.parameters() if p.requires_grad)
resnet34_params = sum(p.numel() for p in resnet34.parameters() if p.requires_grad)
print(f"Number of trainable parameters in ResNet18: {resnet18_params:,}")
print(f"Number of trainable parameters in ResNet34: {resnet34_params:,}")

Number of trainable parameters in ResNet18: 3,851,397
Number of trainable parameters in ResNet34: 7,225,733


In [6]:
from train_eval.trainer import Trainer
import torch
from torch import optim

criterion = torch.nn.CrossEntropyLoss()

learning_rate = 0.002
weight_decay = 0.00001

optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate, weight_decay=weight_decay)

trainer = Trainer(optimizer, criterion)
